In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from transformator.context import Context
from transformator.lambda_tree import LambdaTree

In [47]:
PROBLEM_N = 6
# PROBLEM_N = 5
# PROBLEM_N = 4
REMOVE_POINTERS = False

In [48]:
c = Context()

In [49]:
c.ns = list(range(PROBLEM_N-4, PROBLEM_N+1))
c.class_constructor = lambda n: LambdaTree.generate(n)

In [50]:
c.section_expressions = [
    [
        ("n * Cn", "llss"),
    ],
    [
        ("2 * (2 * n + 1) * Cn", "aosu"),
        ("- (2 * n) * Cn", "lls"),
        ("- (2 * n + 1) * Cn", "lss"),
    ],
    [
        ("- (4 * n) * Cn", "aou"),
        ("(n) * Cn", "ll"),
        ("(4 * n + 2) * Cn", "ls"),
        ("(n + 1) * Cn", "ss"),
    ],
    [
        ("- (2 * n + 1) * Cn", "l"),
        ("- 2 * (n + 1) * Cn", "s"),
    ],
    [
        ("(n + 1) * Cn", ""),
    ],
]

In [51]:
c.section_counts

[[4], [56, -24, -28], [-144, 36, 162, 45], [-242, -264], [399]]

In [52]:
c.classes_counts

[4, 4, 99, -506, 399]

In [53]:
sum(c.classes_counts)

0

In [54]:
c.clear_all_classes()

In [55]:
c.define_class(4)

c.define_section(0)

c.append_r_class_subtree_visitor(1,
    lambda kind, left, right:
        None if kind[0] is not None else
        [kind + ("*", )] + left + right
)

c.print_stats()

# define 4,0: (n + 1) * Cn []
=== Classes stats ===
[ ]  0,0:     0 of     4 | n * Cn                    [llss]
[ ]  1,0:     0 of    56 | 2 * (2 * n + 1) * Cn      [aosu]
[ ]  1,1:     0 of   -24 | - (2 * n) * Cn            [lls]
[ ]  1,2:     0 of   -28 | - (2 * n + 1) * Cn        [lss]
[ ]  2,0:     0 of  -144 | - (4 * n) * Cn            [aou]
[ ]  2,1:     0 of    36 | (n) * Cn                  [ll]
[ ]  2,2:     0 of   162 | (4 * n + 2) * Cn          [ls]
[ ]  2,3:     0 of    45 | (n + 1) * Cn              [ss]
[ ]  3,0:     0 of  -242 | - (2 * n + 1) * Cn        [l]
[ ]  3,1:     0 of  -264 | - 2 * (n + 1) * Cn        [s]
[x]  4,0:   399 of   399 | (n + 1) * Cn              []
=== Diff stats ===
4,0:     0   399


In [56]:
c.define_class(3)

c.define_section(0)

c.append_r_class_subtree_visitor(-1,
    lambda kind, left, right:
        None if kind[0] is None else
        ["l", kind] + left + right + [(None, "*")]
)

c.append_r_class_right_parent_subtree_visitor(-1,
    lambda kind, parent_kind, left, right:
        None if kind[0] is not None else
        ["l", parent_kind] + [left[0] + ("*",)] + left[1:] + right + [None]
)

c.append_r_class_left_parent_subtree_visitor(-1,
    lambda kind, parent_kind, left, right:
        None if kind[0] is not None else
        ["l", parent_kind] + left + [right[0] + ("*",)] + right[1:] + [None]
)

c.define_section(1)

c.append_r_class_left_parent_subtree_visitor(-1,
    lambda kind, parent_kind, left, right:
        None if kind[0] is not None else
        ["s", None, parent_kind] + left + [right[0] + ("*",)] + right[1:]
)

c.append_r_class_right_parent_subtree_visitor(-1,
    lambda kind, parent_kind, left, right:
        None if kind[0] is not None else
        ["s", None, parent_kind] + [left[0] + ("*",)] + left[1:] + right
)

c.append_r_class_right_parent_subtree_visitor(-1,
    lambda kind, parent_kind, left, right:
        None if kind[0] is not None else
        ["s", (None, "*"), parent_kind] + left + right
)

c.print_stats()

# define 3,0: - (2 * n + 1) * Cn [l]
# define 3,1: - 2 * (n + 1) * Cn [s]
=== Classes stats ===
[ ]  0,0:     0 of     4 | n * Cn                    [llss]
[ ]  1,0:     0 of    56 | 2 * (2 * n + 1) * Cn      [aosu]
[ ]  1,1:     0 of   -24 | - (2 * n) * Cn            [lls]
[ ]  1,2:     0 of   -28 | - (2 * n + 1) * Cn        [lss]
[ ]  2,0:     0 of  -144 | - (4 * n) * Cn            [aou]
[ ]  2,1:     0 of    36 | (n) * Cn                  [ll]
[ ]  2,2:     0 of   162 | (4 * n + 2) * Cn          [ls]
[ ]  2,3:     0 of    45 | (n + 1) * Cn              [ss]
[x]  3,0:  -242 of  -242 | - (2 * n + 1) * Cn        [l]
[ ]  3,1:  -195 of  -264 | - 2 * (n + 1) * Cn        [s]
[x]  4,0:   399 of   399 | (n + 1) * Cn              []
=== Diff stats ===
3,0:    86     0
3,1:    64     0
4,0:     0   112


In [57]:
c.define_class(2)

c.define_section(0)

c.append_r_class_subtree_visitor(-1,
    lambda kind, left, right:
        None if kind[0] is None else
        ["a", "o", (None, "*"), None, kind] + left + right
)

c.append_r_class_subtree_visitor(-1,
    lambda kind, left, right:
        None if kind[0] is None else
        ["a", "o", None, (None, "*"), kind] + left + right
)

c.append_r_class_subtree_visitor(-1,
    lambda kind, left, right:
        None if kind[0] is None else
        ["a", kind] + left + right + ["o", (None, "*"), None]
)

c.append_r_class_subtree_visitor(-1,
    lambda kind, left, right:
        None if kind[0] is None else
        ["a", kind] + left + right + ["o", None, (None, "*")]
)

c.define_section(1)

c.append_r_class_subtree_visitor(1,
    lambda kind, left, right:
        None if kind[0] is None else
        ["l", "l", kind] + left + right + [(None, "*"), None]
)

c.define_section(2)

c.append_r_class_right_parent_subtree_visitor(1,
    lambda kind, parent_kind, left, right:
        None if kind[0] is not None else
        ["l", "s", (None, "*"), parent_kind] + left + right + [None]
)

c.append_r_class_subtree_visitor(1,
    lambda kind, left, right:
        None if kind[0] is None else
        ["s", None, "l", kind] + left + right + [(None, "*")]
)

c.append_r_class_right_parent_subtree_visitor(1,
    lambda kind, parent_kind, left, right:
        None if kind[0] is not None else
        ["s", None, "l", parent_kind] + left + right + [(None, "*")]
)

c.append_r_class_right_parent_subtree_visitor(1,
    lambda kind, parent_kind, left, right:
        None if kind[0] is not None else
        ["s", None, "l", parent_kind] + [left[0] + ("*", )] + left[1:] + right + [None]
)

c.append_r_class_left_parent_subtree_visitor(1,
    lambda kind, parent_kind, left, right:
        None if kind[0] is not None else
        ["s", None, "l", parent_kind] + left + [right[0] + ("*", )] + right[1:] + [None]
)

c.define_section(3)

c.append_r_class_right_parent_subtree_visitor(1,
    lambda kind, parent_kind, left, right:
        None if kind[0] is not None else
        ["s", None, "s", (None, "*"), parent_kind] + left + right
)

c.print_stats()

# define 2,0: - (4 * n) * Cn [aou]
# define 2,1: (n) * Cn [ll]
# define 2,2: (4 * n + 2) * Cn [ls]
# define 2,3: (n + 1) * Cn [ss]
=== Classes stats ===
[ ]  0,0:     0 of     4 | n * Cn                    [llss]
[ ]  1,0:     0 of    56 | 2 * (2 * n + 1) * Cn      [aosu]
[ ]  1,1:     0 of   -24 | - (2 * n) * Cn            [lls]
[ ]  1,2:     0 of   -28 | - (2 * n + 1) * Cn        [lss]
[x]  2,0:  -144 of  -144 | - (4 * n) * Cn            [aou]
[x]  2,1:    36 of    36 | (n) * Cn                  [ll]
[ ]  2,2:   125 of   162 | (4 * n + 2) * Cn          [ls]
[ ]  2,3:    22 of    45 | (n + 1) * Cn              [ss]
[x]  3,0:  -242 of  -242 | - (2 * n + 1) * Cn        [l]
[ ]  3,1:  -195 of  -264 | - 2 * (n + 1) * Cn        [s]
[x]  4,0:   399 of   399 | (n + 1) * Cn              []
=== Diff stats ===
2,0:    32     0
2,1:     0     8
2,2:     0    25


In [73]:
c.define_class(1)

c.define_section(0)

c.define_section(1)

c.append_r_class_right_parent_subtree_visitor(-1,
    lambda kind, parent_kind, left, right:
        None if kind[0] is not None else
        ["s", None, "l", "l", parent_kind] + left + right + [(None, "*"), None]
)

c.append_r_class_right_parent_subtree_visitor(-1,
    lambda kind, parent_kind, left, right:
        ["s", None, "l", "l", parent_kind] + left + right + [(None, "*"), None]
)


c.define_section(2)

c.append_r_class_right_parent_subtree_visitor(-1,
    lambda kind, parent_kind, left, right:
        None if kind[0] is not None else
        ["s", None, "s", None, "l", parent_kind] + left + right + [(None, "*")]
)

c.append_r_class_left_parent_subtree_visitor(-1,
    lambda kind, parent_kind, left, right:
        ["s", None, "l", "s", (None, "*"), parent_kind] + left + right + [None]
)

c.print_stats()

# define 1,0: 2 * (2 * n + 1) * Cn [aosu]
# define 1,1: - (2 * n) * Cn [lls]
# define 1,2: - (2 * n + 1) * Cn [lss]
=== Classes stats ===
[ ]  0,0:     0 of     4 | n * Cn                    [llss]
[ ]  1,0:     0 of    56 | 2 * (2 * n + 1) * Cn      [aosu]
[ ]  1,1:   -20 of   -24 | - (2 * n) * Cn            [lls]
[ ]  1,2:   -20 of   -28 | - (2 * n + 1) * Cn        [lss]
[x]  2,0:  -144 of  -144 | - (4 * n) * Cn            [aou]
[x]  2,1:    36 of    36 | (n) * Cn                  [ll]
[ ]  2,2:   125 of   162 | (4 * n + 2) * Cn          [ls]
[ ]  2,3:    22 of    45 | (n + 1) * Cn              [ss]
[x]  3,0:  -242 of  -242 | - (2 * n + 1) * Cn        [l]
[ ]  3,1:  -195 of  -264 | - 2 * (n + 1) * Cn        [s]
[x]  4,0:   399 of   399 | (n + 1) * Cn              []
=== Diff stats ===
2,0:    32     0
1,2:     4     0
1,1:     3     0


In [81]:
c.define_class(0)

c.define_section(0)


c.print_stats()

# define 0,0: n * Cn [llss]
=== Classes stats ===
[ ]  0,0:     0 of     4 | n * Cn                    [llss]
[ ]  1,0:     0 of    56 | 2 * (2 * n + 1) * Cn      [aosu]
[ ]  1,1:   -20 of   -24 | - (2 * n) * Cn            [lls]
[ ]  1,2:   -20 of   -28 | - (2 * n + 1) * Cn        [lss]
[x]  2,0:  -144 of  -144 | - (4 * n) * Cn            [aou]
[x]  2,1:    36 of    36 | (n) * Cn                  [ll]
[ ]  2,2:   125 of   162 | (4 * n + 2) * Cn          [ls]
[ ]  2,3:    22 of    45 | (n + 1) * Cn              [ss]
[x]  3,0:  -242 of  -242 | - (2 * n + 1) * Cn        [l]
[ ]  3,1:  -195 of  -264 | - 2 * (n + 1) * Cn        [s]
[x]  4,0:   399 of   399 | (n + 1) * Cn              []
=== Diff stats ===
2,0:    32     0
1,2:     4     0
1,1:     3     0


In [83]:
c.display_diff_trees(2, 2)

In [84]:
c.display_trees(1, 2, -1)

"<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?> <!DOCTYPE svg PUBLIC ""-//W3C//DTD SVG 1.1//EN"" ""http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd""> <!-- Generated by graphviz version 2.44.1 (20200629.0846) --> <!-- Pages: 1 --> <!-- 0 --> 0 l <!-- 1 --> 1 l <!-- 0--1 --> 0--1 <!-- 6 --> 6 <!-- 0--6 --> 0--6 <!-- 2 --> 2 o <!-- 1--2 --> 1--2 <!-- 5 --> 5 <!-- 1--5 --> 1--5 <!-- 3 --> 3 <!-- 2--3 --> 2--3 <!-- 4 --> 4 <!-- 2--4 --> 2--4"


"<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?> <!DOCTYPE svg PUBLIC ""-//W3C//DTD SVG 1.1//EN"" ""http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd""> <!-- Generated by graphviz version 2.44.1 (20200629.0846) --> <!-- Pages: 1 --> <!-- 0 --> 0 l <!-- 1 --> 1 l <!-- 0--1 --> 0--1 <!-- 6 --> 6 <!-- 0--6 --> 0--6 <!-- 2 --> 2 o <!-- 1--2 --> 1--2 <!-- 5 --> 5 <!-- 1--5 --> 1--5 <!-- 3 --> 3 <!-- 2--3 --> 2--3 <!-- 4 --> 4 <!-- 2--4 --> 2--4"


"<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?> <!DOCTYPE svg PUBLIC ""-//W3C//DTD SVG 1.1//EN"" ""http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd""> <!-- Generated by graphviz version 2.44.1 (20200629.0846) --> <!-- Pages: 1 --> <!-- 0 --> 0 l <!-- 1 --> 1 l <!-- 0--1 --> 0--1 <!-- 6 --> 6 <!-- 0--6 --> 0--6 <!-- 2 --> 2 o <!-- 1--2 --> 1--2 <!-- 5 --> 5 <!-- 1--5 --> 1--5 <!-- 3 --> 3 <!-- 2--3 --> 2--3 <!-- 4 --> 4 <!-- 2--4 --> 2--4"


"<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?> <!DOCTYPE svg PUBLIC ""-//W3C//DTD SVG 1.1//EN"" ""http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd""> <!-- Generated by graphviz version 2.44.1 (20200629.0846) --> <!-- Pages: 1 --> <!-- 0 --> 0 l <!-- 1 --> 1 s <!-- 0--1 --> 0--1 <!-- 6 --> 6 <!-- 0--6 --> 0--6 <!-- 2 --> 2 <!-- 1--2 --> 1--2 <!-- 3 --> 3 o <!-- 1--3 --> 1--3 <!-- 4 --> 4 <!-- 3--4 --> 3--4 <!-- 5 --> 5 <!-- 3--5 --> 3--5"


"<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?> <!DOCTYPE svg PUBLIC ""-//W3C//DTD SVG 1.1//EN"" ""http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd""> <!-- Generated by graphviz version 2.44.1 (20200629.0846) --> <!-- Pages: 1 --> <!-- 0 --> 0 l <!-- 1 --> 1 s <!-- 0--1 --> 0--1 <!-- 6 --> 6 <!-- 0--6 --> 0--6 <!-- 2 --> 2 <!-- 1--2 --> 1--2 <!-- 3 --> 3 o <!-- 1--3 --> 1--3 <!-- 4 --> 4 <!-- 3--4 --> 3--4 <!-- 5 --> 5 <!-- 3--5 --> 3--5"


...

"<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?> <!DOCTYPE svg PUBLIC ""-//W3C//DTD SVG 1.1//EN"" ""http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd""> <!-- Generated by graphviz version 2.44.1 (20200629.0846) --> <!-- Pages: 1 --> <!-- 0 --> 0 a <!-- 1 --> 1 o <!-- 0--1 --> 0--1 <!-- 4 --> 4 o <!-- 0--4 --> 0--4 <!-- 2 --> 2 <!-- 1--2 --> 1--2 <!-- 3 --> 3 <!-- 1--3 --> 1--3 <!-- 5 --> 5 <!-- 4--5 --> 4--5 <!-- 6 --> 6 <!-- 4--6 --> 4--6"


"<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?> <!DOCTYPE svg PUBLIC ""-//W3C//DTD SVG 1.1//EN"" ""http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd""> <!-- Generated by graphviz version 2.44.1 (20200629.0846) --> <!-- Pages: 1 --> <!-- 0 --> 0 a <!-- 1 --> 1 o <!-- 0--1 --> 0--1 <!-- 4 --> 4 o <!-- 0--4 --> 0--4 <!-- 2 --> 2 <!-- 1--2 --> 1--2 <!-- 3 --> 3 <!-- 1--3 --> 1--3 <!-- 5 --> 5 <!-- 4--5 --> 4--5 <!-- 6 --> 6 <!-- 4--6 --> 4--6"


"<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?> <!DOCTYPE svg PUBLIC ""-//W3C//DTD SVG 1.1//EN"" ""http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd""> <!-- Generated by graphviz version 2.44.1 (20200629.0846) --> <!-- Pages: 1 --> <!-- 0 --> 0 s <!-- 1 --> 1 <!-- 0--1 --> 0--1 <!-- 2 --> 2 s <!-- 0--2 --> 0--2 <!-- 3 --> 3 <!-- 2--3 --> 2--3 <!-- 4 --> 4 o <!-- 2--4 --> 2--4 <!-- 5 --> 5 <!-- 4--5 --> 4--5 <!-- 6 --> 6 <!-- 4--6 --> 4--6"


"<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?> <!DOCTYPE svg PUBLIC ""-//W3C//DTD SVG 1.1//EN"" ""http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd""> <!-- Generated by graphviz version 2.44.1 (20200629.0846) --> <!-- Pages: 1 --> <!-- 0 --> 0 s <!-- 1 --> 1 <!-- 0--1 --> 0--1 <!-- 2 --> 2 s <!-- 0--2 --> 0--2 <!-- 3 --> 3 <!-- 2--3 --> 2--3 <!-- 4 --> 4 o <!-- 2--4 --> 2--4 <!-- 5 --> 5 <!-- 4--5 --> 4--5 <!-- 6 --> 6 <!-- 4--6 --> 4--6"


"<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?> <!DOCTYPE svg PUBLIC ""-//W3C//DTD SVG 1.1//EN"" ""http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd""> <!-- Generated by graphviz version 2.44.1 (20200629.0846) --> <!-- Pages: 1 --> <!-- 0 --> 0 s <!-- 1 --> 1 <!-- 0--1 --> 0--1 <!-- 2 --> 2 s <!-- 0--2 --> 0--2 <!-- 3 --> 3 <!-- 2--3 --> 2--3 <!-- 4 --> 4 o <!-- 2--4 --> 2--4 <!-- 5 --> 5 <!-- 4--5 --> 4--5 <!-- 6 --> 6 <!-- 4--6 --> 4--6"


In [27]:
c.display_trees(1, 1, -1)


"<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?> <!DOCTYPE svg PUBLIC ""-//W3C//DTD SVG 1.1//EN"" ""http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd""> <!-- Generated by graphviz version 2.44.1 (20200629.0846) --> <!-- Pages: 1 --> <!-- 0 --> 0 l <!-- 1 --> 1 s <!-- 0--1 --> 0--1 <!-- 10 --> 10 <!-- 0--10 --> 0--10 <!-- 2 --> 2 <!-- 1--2 --> 1--2 <!-- 3 --> 3 l <!-- 1--3 --> 1--3 <!-- 4 --> 4 l <!-- 3--4 --> 3--4 <!-- 9 --> 9 <!-- 3--9 --> 3--9 <!-- 5 --> 5 o <!-- 4--5 --> 4--5 <!-- 8 --> 8 * <!-- 4--8 --> 4--8 <!-- 6 --> 6 <!-- 5--6 --> 5--6 <!-- 7 --> 7 <!-- 5--7 --> 5--7","<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?> <!DOCTYPE svg PUBLIC ""-//W3C//DTD SVG 1.1//EN"" ""http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd""> <!-- Generated by graphviz version 2.44.1 (20200629.0846) --> <!-- Pages: 1 --> <!-- 0 --> 0 l <!-- 1 --> 1 o <!-- 0--1 --> 0--1 <!-- 4 --> 4 <!-- 0--4 --> 0--4 <!-- 2 --> 2 <!-- 1--2 --> 1--2 <!-- 3 --> 3 <!-- 1--3 --> 1--3"


"<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?> <!DOCTYPE svg PUBLIC ""-//W3C//DTD SVG 1.1//EN"" ""http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd""> <!-- Generated by graphviz version 2.44.1 (20200629.0846) --> <!-- Pages: 1 --> <!-- 0 --> 0 s <!-- 1 --> 1 <!-- 0--1 --> 0--1 <!-- 2 --> 2 l <!-- 0--2 --> 0--2 <!-- 3 --> 3 l <!-- 2--3 --> 2--3 <!-- 10 --> 10 <!-- 2--10 --> 2--10 <!-- 4 --> 4 s <!-- 3--4 --> 3--4 <!-- 9 --> 9 * <!-- 3--9 --> 3--9 <!-- 5 --> 5 <!-- 4--5 --> 4--5 <!-- 6 --> 6 o <!-- 4--6 --> 4--6 <!-- 7 --> 7 <!-- 6--7 --> 6--7 <!-- 8 --> 8 <!-- 6--8 --> 6--8","<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?> <!DOCTYPE svg PUBLIC ""-//W3C//DTD SVG 1.1//EN"" ""http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd""> <!-- Generated by graphviz version 2.44.1 (20200629.0846) --> <!-- Pages: 1 --> <!-- 0 --> 0 s <!-- 1 --> 1 <!-- 0--1 --> 0--1 <!-- 2 --> 2 o <!-- 0--2 --> 0--2 <!-- 3 --> 3 <!-- 2--3 --> 2--3 <!-- 4 --> 4 <!-- 2--4 --> 2--4"


"<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?> <!DOCTYPE svg PUBLIC ""-//W3C//DTD SVG 1.1//EN"" ""http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd""> <!-- Generated by graphviz version 2.44.1 (20200629.0846) --> <!-- Pages: 1 --> <!-- 0 --> 0 s <!-- 1 --> 1 <!-- 0--1 --> 0--1 <!-- 2 --> 2 s <!-- 0--2 --> 0--2 <!-- 3 --> 3 <!-- 2--3 --> 2--3 <!-- 4 --> 4 l <!-- 2--4 --> 2--4 <!-- 5 --> 5 l <!-- 4--5 --> 4--5 <!-- 10 --> 10 <!-- 4--10 --> 4--10 <!-- 6 --> 6 o <!-- 5--6 --> 5--6 <!-- 9 --> 9 * <!-- 5--9 --> 5--9 <!-- 7 --> 7 <!-- 6--7 --> 6--7 <!-- 8 --> 8 <!-- 6--8 --> 6--8","<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?> <!DOCTYPE svg PUBLIC ""-//W3C//DTD SVG 1.1//EN"" ""http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd""> <!-- Generated by graphviz version 2.44.1 (20200629.0846) --> <!-- Pages: 1 --> <!-- 0 --> 0 s <!-- 1 --> 1 <!-- 0--1 --> 0--1 <!-- 2 --> 2 o <!-- 0--2 --> 0--2 <!-- 3 --> 3 <!-- 2--3 --> 2--3 <!-- 4 --> 4 <!-- 2--4 --> 2--4"
